<a href="https://colab.research.google.com/github/Aftabalamrana/CT5102/blob/master/KNN_with_Multiple_Labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Goal:** 
To predict the location of a company based on historical data

# Imports & Load Data

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
# !pip install reverse_geocoder
# import libraries
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
# import seaborn as sns
# import reverse_geocoder as revgc

In [0]:
seed = 10
np.random.seed(seed)
pd.set_option('display.expand_frame_repr', False) # to display all columns

**Dataset Information (columns)**: 
1. **LAT**:
2. **LNG**:
3. **COUNT**: always 1
4. **COMPANY_STATUS_YM**:
5. **COMPANY_REG_YM**:
6. **COMPANY_NUM**: unique for all rows (pk)
7. **COMPANY_NAME**:  unique for all rows (pk)
8. **ADDRESS**:
9. **COUNTY**:
10. **PHONE**: optional field, mostly null
11. **WEBSITE**: optional field, mostly null
12. **COMPANY_STATUS_CODE**:
13. **COMPANY_TYPE_CODE**:
14. **SIZE**: 0,1,2,3,4
15. **EMPLOYEE**: optional field, mostly null
16. **INDUSTRY**: around 800 indusltries

In [56]:
# Load data all data
original_df = pd.read_csv('/content/ml_data.csv') 
print('original dataframe shape: ', original_df.shape)

original_df.head(1)

original dataframe shape:  (64826, 16)


,LAT,LNG,COUNT,COMPANY_STATUS_YM,COMPANY_REG_YM,COMPANY_NUM,COMPANY_NAME,ADDRESS,COUNTY,PHONE,WEBSITE,COMPANY_STATUS_CODE,COMPANY_TYPE_CODE,SIZE,EMPLOYEE,INDUSTRY
0,54.763626,-8.410592,1,199001,199901,4929,ARDARA WOOLLLEN AND INDUSTRIAL COMPANY LIMITED,"Ardara, co donegal",donegal,NaN,NaN,8,0,4,NaN,unknown


# Exploring Data

In [0]:
# print(original_df.isna().sum()) # Check missing values

In [58]:
# drop some fields for now
# Remove missing values IF AVAILABLE and print first 10 samples
# df = df.dropna()
# print(df.shape)
df = original_df.drop([
              # 'LAT', 
              # 'LNG',
              'COUNT',
              'COMPANY_STATUS_YM', 
              'COMPANY_REG_YM',
              'COMPANY_NUM', 
              'COMPANY_NAME',
              # 'ADDRESS',
              # 'COUNTY',
              'PHONE',
              'WEBSITE',
              # 'COMPANY_STATUS_CODE',
              # 'COMPANY_TYPE_CODE',
              # 'SIZE',
              'EMPLOYEE',
              # 'INDUSTRY'
              ], axis=1)

# remove where size or industry is unknow
# df = df.loc[(df['SIZE'] != 4) & (df['INDUSTRY'] != 'unknown')]
# df = df.loc[(df['SIZE'] != 4)] # size === 4 is unknow IT will be change in next version of data
# df = df.loc[(df['INDUSTRY'] != 'unknown')]
print(df.shape)
df.head(3)

(64826, 8)


,LAT,LNG,ADDRESS,COUNTY,COMPANY_STATUS_CODE,COMPANY_TYPE_CODE,SIZE,INDUSTRY
0,54.763626,-8.410592,"Ardara, co donegal",donegal,8,0,4,unknown
1,54.486674,-8.273761,"Finner avenue, bundoran, co. donegal.",donegal,8,1,4,financial intermediation except insurance and ...
2,55.135246,-7.453581,"8, swilly gardens, buncrana, co. donegal.",donegal,8,1,4,general mechanical engineering


In [59]:
le = preprocessing.LabelEncoder()
df['ADDRESS'] = le.fit_transform(df['ADDRESS'])
df['COUNTY'] = le.fit_transform(df['COUNTY'])
df['INDUSTRY'] = le.fit_transform(df['INDUSTRY'])
print(df.dtypes)
df.head(3)

LAT                    float64
LNG                    float64
ADDRESS                  int64
COUNTY                   int64
COMPANY_STATUS_CODE      int64
COMPANY_TYPE_CODE        int64
SIZE                     int64
INDUSTRY                 int64
dtype: object


,LAT,LNG,ADDRESS,COUNTY,COMPANY_STATUS_CODE,COMPANY_TYPE_CODE,SIZE,INDUSTRY
0,54.763626,-8.410592,10742,1,8,0,4,734
1,54.486674,-8.273761,26126,1,8,1,4,122
2,55.135246,-7.453581,9281,1,8,1,4,144


# Select feature

In [60]:
# We now split our dataset into input features (X) and the feature we wish to predict (Y).
target = 'COUNTY'

X = df.drop([target], axis=1).values
X = df.drop([target], axis=1)
Y = df[target].values

print(df.dtypes)
print(X.shape)
print(Y.shape)

LAT                    float64
LNG                    float64
ADDRESS                  int64
COUNTY                   int64
COMPANY_STATUS_CODE      int64
COMPANY_TYPE_CODE        int64
SIZE                     int64
INDUSTRY                 int64
dtype: object
(64826, 7)
(64826,)


In [61]:
# Statistical summary of the variables
print(X.describe())

                LAT           LNG       ADDRESS  COMPANY_STATUS_CODE  COMPANY_TYPE_CODE          SIZE      INDUSTRY
count  64826.000000  64826.000000  64826.000000         64826.000000       64826.000000  64826.000000  64826.000000
mean      53.694364     -8.696442  23848.397819             4.717305          36.501373      2.201586    338.834958
std        0.699855      0.568375  13181.155768             3.517527          18.032037      1.925098    242.670274
min       51.509458    -10.251647      0.000000             1.000000           0.000000      0.000000      0.000000
25%       53.268729     -9.050553  12522.000000             1.000000          19.000000      0.000000    114.000000
50%       53.512564     -8.878837  24331.000000             8.000000          48.000000      4.000000    374.000000
75%       54.122070     -8.314852  35032.750000             8.000000          53.000000      4.000000    550.000000
max       55.374781     -6.123676  46470.000000            25.000000    

**Scale of all the variables is different. Therefore, feature scaling is important?????**

In [62]:
# Check for class imbalance
print(df.groupby(Y).size())

0    12648
1    10925
2    22459
3     2186
4     7926
5     3896
6     4786
dtype: int64


In [63]:
# data normalization to make sure that the scale of the data is similar for the proper initialization of the NN
# Normalize features within range 0 (minimum) and 1 (maximum)
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
X = pd.DataFrame(X)
X.head(3)

,0,1,2,3,4,5,6
0,0.841887,0.445995,0.231160,0.291667,0.000000,1.0,0.929114
1,0.770237,0.479143,0.562212,0.291667,0.015873,1.0,0.154430
2,0.938030,0.677831,0.199720,0.291667,0.015873,1.0,0.182278


In [64]:
# Y = preprocessing.scale(Y)
# for a in Y:
#   print(a)
np.unique(Y)

array([0, 1, 2, 3, 4, 5, 6])

In [65]:
# Convert target Y to one hot encoded Y for Neural Network
# Y = pd.get_dummies(Y)
# If target is in string form, use following code:
# First encode target values as integers from string
# Then perform one hot encoding

encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
Y = np_utils.to_categorical(Y)

Y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
# For Keras, convert dataframe to array values (Inbuilt requirement of Keras)
# X = X.values
# Y = Y.values

In [0]:
# https://www.datacamp.com/community/tutorials/k-nearest-neighbor-classification-scikit-learn
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3) # 70% training and 30% test

# Model

In [68]:
#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=7)

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8973159193747429
